### Create the dataset dir


In [11]:
os.listdir("cats")

['state.json', 'dataset_info.json', 'data-00000-of-00001.arrow']

#### Add a new image to dataset

In [12]:
from datasets import Dataset, concatenate_datasets
import shutil
import os

output_directory = './cats'
existing_dataset = Dataset.load_from_disk(output_directory)
new_image_path = 'cat.jpeg'

# Create a new dataset entry for the new image
new_entry = Dataset.from_list([{'image': new_image_path}], features=existing_dataset.features)
updated_dataset = concatenate_datasets([existing_dataset, new_entry])

# Save the updated dataset to a temporary directory because of permissions
temp_directory = './cats_temp'
updated_dataset.save_to_disk(temp_directory)

# Replace the original directory with the updated dataset
shutil.rmtree(output_directory)  # Delete the original directory
shutil.move(temp_directory, output_directory)  # Move the temp directory to the original path

# Print the length to verify the new image was added
print(f"Updated dataset length: {len(updated_dataset)}")


Saving the dataset (1/1 shards): 100%|██████████| 4/4 [00:00<00:00, 1030.60 examples/s]

Updated dataset length: 4


# Make a prediction with ResNet

In [13]:
from transformers import AutoImageProcessor, ResNetForImageClassification
import torch
from datasets import load_from_disk

# dataset = load_dataset("huggingface/cats-image")
ds = load_from_disk('./cats')
image = ds[1]['image']

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

inputs = processor(image, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# model predicts one of the 1000 ImageNet classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

AttributeError: module 'PIL.Image' has no attribute 'ExifTags'

In [50]:
type(model)

transformers.models.resnet.modeling_resnet.ResNetForImageClassification